# Constrained minimization example

We consider the simple example from the <b>Gloptipoly</b> manual,

\begin{array}{ll}
\text{minimize} & -2 x_1 + x_2 - x_3\\
\text{subject to} & 24 - 20 x_1 + 9 x_2 - 13 x_3 + 4 x_1^2 - 4 x_1 x_2 + 4 x_1 x_3 + 2 x_2^2 - 2 x_2 x_3 + 2 x_3^2 \geq 0\\
& x_1 + x_2 + x_3 \leq 4, \quad 3 x_2 \leq 6\\
& 0 \leq x_1 \leq 2, \quad 0 \leq x_2, \quad 0\leq x_3\leq 3
\end{array}

In [1]:
using Polyopt

We define the variables

In [2]:
x1, x2, x3 = variables(["x1","x2","x3"]);

and the objective function

In [3]:
f = -2*x1 + x2 - x3;

and an array with all inequalities in the form $g_i(x) \geq 0$,

In [4]:
g = [24 - 20*x1 + 9*x2 - 13*x3 + 4*x1^2 - 4*x1*x2 + 4*x1*x3 + 2*x2^2 - 2*x2*x3 + 2*x3^2,
     4 - (x1 + x2 + x3),
     6 - (3*x2 + x3),
     x1, 2-x1, 
     x2, 
     x3, 3-x3]

8-element Array{Polyopt.Poly{Int64},1}:
 24-13*x3+2*x3^2+9*x2-2*x2*x3+2*x2^2-20*x1+4*x1*x3-4*x1*x2+4*x1^2
 4-x3-x2-x1                                                      
 6-x3-3*x2                                                       
 x1                                                              
 2-x1                                                            
 x2                                                              
 x3                                                              
 3-x3                                                            

## Solving the second-order moment relaxation

This problem has multiple optimal minima, so we perturb the problem to locate one of the minimizers,

In [5]:
prob = momentprob(2, f + 1e-3*(x1+x2+x3), g);

We next solve the problem using MOSEK, turning off log-output:

In [6]:
X, Z, t, y, solsta = solve_mosek(prob, showlog=false);

In [7]:
t

-5.688354583203682

## Solving the third-order moment relaxation

In [8]:
prob = momentprob(3, f + 1e-3*(x1+x2+x3), g);

In [9]:
X, Z, t, y, solsta = solve_mosek(prob, showlog=false);

The lower bound decreases going from order two to order three,

In [10]:
t

-4.064806932586617

## Solving the fourth-order order moment relaxation

In [11]:
prob = momentprob(4, f + 1e-3*(x1+x2+x3), g);

We solve the fourth-order relaxation, this time showing the log-output from the solver,

In [12]:
X, Z, t, y, solsta = solve_mosek(prob, showlog=true);

Open file 'polyopt.task'
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 165             
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 9               
  Integer variables      : 0               

Optimizer started.
Conic interior-point optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator - tries                  : 0                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Optimizer  - threads                : 4               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 165
Optimiz

Again the lower increases when we increase the relaxation order,

In [13]:
t

-3.997999988220454

At this point the hierarchy of relaxations has converged; higher order relaxations give the same bound. Let us inspect the recovered solution

In [14]:
[prob.basis y]

165×2 Array{Polyopt.Poly{Float64},2}:
 1.0           1.0        
 x3            7.95982e-6 
 x3^2          2.38727e-5 
 x3^3          7.16109e-5 
 x3^4          0.000214822
 x3^5          0.000644446
 x3^6          0.0019333  
 x3^7          0.0057998  
 x3^8          14.2424    
 x2            6.80074e-9 
 x2*x3         1.22965e-8 
 x2*x3^2       3.30135e-8 
 x2*x3^3       9.46562e-8 
 ⋮                        
 x1^5*x2^2*x3  -0.566143  
 x1^5*x2^3     1.37997    
 x1^6          63.9998    
 x1^6*x3       1.96216e-7 
 x1^6*x3^2     3.22142    
 x1^6*x2       1.2041e-7  
 x1^6*x2*x3    -0.344823  
 x1^6*x2^2     3.18658    
 x1^7          128.0      
 x1^7*x3       -2.09519   
 x1^7*x2       2.03974    
 x1^8          266.08     

Let us verify that the recovered solution

In [15]:
X1, X2, X3 = Polyopt.vectorize([x1,x2,x3], 8)*y

3-element Array{Float64,1}:
 2.0       
 6.80074e-9
 7.95982e-6

 is optimal. It achieves the same objective value as the relaxation,

In [16]:
Polyopt.evalpoly(f, [X1; X2; X3])

-3.9999999922512517

and it is feasible,

In [17]:
[ Polyopt.evalpoly(gi, [X1; X2; X3]) for gi=g ]

8-element Array{Float64,1}:
 -2.38707e-5
  2.0       
  5.99999   
  2.0       
  3.98038e-6
  6.80074e-9
  7.95982e-6
  2.99999   